# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

import sys


from data_wrangling import general_utils, specific_utils, process_monkey_information, base_processing_class, retrieve_raw_data, further_processing_class
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils, plot_cluster_replacement
from decision_making_analysis.decision_making import decision_making_utils, plot_decision_making, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_helper_class, GUAT_collect_info_class, GUAT_combine_info_class, add_features_GUAT_and_TAFT
from decision_making_analysis import free_selection, replacement, trajectory_info
from visualization.matplotlib_tools import plot_trials, plot_polar, additional_plots, plot_behaviors_utils, plot_statistics, monkey_heading_utils
from visualization.animation import animation_func, animation_utils, animation_class
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from reinforcement_learning.base_classes import env_utils, base_env, more_envs, rl_base_class, rl_base_utils
from reinforcement_learning.agents.rnn import gru_utils, lstm_utils
from reinforcement_learning.agents.feedforward import interpret_neural_network, sb3_class, sb3_utils
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import seaborn as sns
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, hamming_loss, multilabel_confusion_matrix, fbeta_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from math import pi
from scipy.ndimage import gaussian_filter1d
from importlib import reload

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 101



# Basic data

## Retrieve monkey data

### short version

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = animation_class.AnimationClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data(exists_ok=True, min_distance_to_calculate_angle=10)
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()

In [ ]:
data_item.make_or_retrieve_target_clust_last_vis_df()
target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']

data_item.make_curvature_df()
data_item.make_or_retrieve_best_arc_df()
data_item.null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(data_item.best_arc_df, data_item.monkey_information, column_for_color=None, ignore_error=True)


### long version

In [ ]:
PLAYER = "monkey"
trial_total_num = 2

raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = animation_class.AnimationClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data(exists_ok=True, min_distance_to_calculate_angle=10)
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_new = data_item.ff_caught_T_new
caught_ff_num = len(ff_caught_T_new)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index

data_item.make_or_retrieve_target_clust_last_vis_df()
target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']

data_item.make_curvature_df()
data_item.make_or_retrieve_best_arc_df()
data_item.null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(data_item.best_arc_df, data_item.monkey_information, column_for_color=None, ignore_error=True)


data_item.make_or_retrieve_all_trial_patterns(exists_ok=True)
data_item.make_or_retrieve_pattern_frequencies(exists_ok=True)
data_item.make_or_retrieve_all_trial_features(exists_ok=True)
data_item.make_or_retrieve_feature_statistics(exists_ok=True)
# data_item.make_info_of_monkey()
# data_item.make_or_retrieve_target_closest()
# data_item.make_or_retrieve_target_angle_smallest()

# target_closest = data_item.target_closest
# target_angle_smallest = data_item.target_angle_smallest
# all_trial_patterns = data_item.all_trial_patterns
# all_trial_features = data_item.all_trial_features
# pattern_frequencies = data_item.pattern_frequencies
# feature_statistics = data_item.feature_statistics
# info_of_monkey = data_item.info_of_monkey

# data_item.make_PlotTrials_args()
# data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)

PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_new)

plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_new,
                    ff_flash_sorted,)


classic_plot_kwargs = {'player': PLAYER,
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'show_eye_positions': True,
                       'show_eye_positions_on_the_right': True,
                       'show_connect_path_eye_positions': True,
                       #=== below is different from animation_plot_kwargs
                       'hitting_arena_edge_ok': False,
                       'trial_too_short_ok': False}

combined_plot_kwargs = {'player': PLAYER,
                        'combined_plot': True,
                        'show_alive_fireflies': False,
                        'show_title': False,
                        'show_start': False}

plot_chunks_kwargs = {'player': PLAYER,
                      'show_stops': True,
                      'show_believed_target_positions': True,
                      'show_reward_boundary': True,
                      'show_scale_bar': True,
                      'hitting_arena_edge_ok': True,
                      'trial_too_short_ok': True,
                      'show_connect_path_ff': True,
                      'trail_color_var': 'abs_ddw',
                      'show_colorbar': True}


plot_polar_kwargs = {'rmax': 400,
                    'show_visible_ff': True,
                    'hitting_arena_edge_ok': True,
                    'return_axes': True,
                    'colors_show_overall_time': True,
                    'show_ff_in_memory': True,
                    'show_target_at_being_caught': True
                    }

animation_plot_kwargs = {
    **classic_plot_kwargs,
    'as_part_of_animation': True,
    'show_eye_positions_on_the_right': False,
    'hitting_arena_edge_ok': True,
    'trial_too_short_ok': True,
    'images_dir': None
}
  
data_item.make_PlotTrials_args()
data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)


plotting_kwargs = {'player': 'monkey',
 'show_stops': True,
 'show_believed_target_positions': True,
 'show_reward_boundary': True,
 'show_scale_bar': True,
 'hitting_arena_edge_ok': True,
 'trial_too_short_ok': True,
 'show_connect_path_ff': True,
 'vary_color_for_connecting_path_ff': True,
 'show_points_when_ff_stop_being_visible': False,
 #'show_connect_path_ff_memory': True,
 'show_alive_fireflies': False,
 'show_visible_fireflies': True,
 'show_in_memory_fireflies': True,
 'connect_path_ff_max_distance': 400,
 #'show_connect_path_ff_except_targets': True,
 'adjust_xy_limits': True,
 'show_null_agent_trajectory': True,
 'show_only_ff_that_monkey_has_passed_by_closely': True,
 'show_null_trajectory_reaching_boundary_ok': False,
 'vary_color_for_connecting_path_ff': False}

plotting_kwargs['null_arc_info_for_plotting'] = data_item.null_arc_info_for_plotting



# Plot statistics (monkey vs agents)

## Calculate statistics of patterns

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0314"
data_item = animation_class.AnimationClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)

data_item.find_patterns()
data_item.make_PlotTrials_args()
data_item.make_df_related_to_patterns_and_features()

## Import and merging stored data

In [ ]:
# The original intention of the code is that pattern_frequencies_a and feature_statistics_a will be freshly generated
# But now, for the sake of demonstration...
pattern_frequencies_a = data_item.pattern_frequencies.copy()
pattern_frequencies_m = data_item.pattern_frequencies.copy()
feature_statistics_a = data_item.feature_statistics.copy()
feature_statistics_m = data_item.feature_statistics.copy()
pattern_frequencies_a['Player'] = "Agent"
feature_statistics_a['Player'] = "Agent"

In [ ]:
agent_monkey_pattern_frequencies = organize_patterns_and_features.combine_df_of_agent_and_monkey(pattern_frequencies_m, pattern_frequencies_a, agent_names = ["Agent", "Agent2", "Agent3"])
agent_monkey_feature_statistics = organize_patterns_and_features.combine_df_of_agent_and_monkey(feature_statistics_m, feature_statistics_a, agent_names = ["Agent", "Agent2", "Agent3"]) 

## Plotting: same plot

In [ ]:
agent_monkey_feature_statistics

In [ ]:
plot_statistics.plot_merged_df(agent_monkey_pattern_frequencies, x="Label", y="Rate")
plt.show()

temp_agent_monkey_feature_statistics = agent_monkey_feature_statistics.copy()
temp_agent_monkey_feature_statistics = temp_agent_monkey_feature_statistics[temp_agent_monkey_feature_statistics['label']!='distance target last seen']
plot_statistics.plot_merged_df(temp_agent_monkey_feature_statistics, x="Label", y="Median")
plt.show()


## Plotting: individual plots

In [ ]:
reload(plot_statistics)

In [ ]:
plot_statistics.plot_merged_df_by_category(agent_monkey_feature_statistics, category_column_name="Label for median", y="Median", category_order=None, percentage=False)
plt.show()


## Plotting histograms

In [ ]:
data_item.make_or_retrieve_all_trial_patterns(exists_ok=True)
data_item.make_or_retrieve_pattern_frequencies(exists_ok=True)
data_item.make_or_retrieve_all_trial_features(exists_ok=True)
data_item.make_or_retrieve_feature_statistics(exists_ok=True)

In [ ]:
# all_trial_features_m means all_trial_features for monkey
all_trial_features_m = data_item.all_trial_features.copy()
# all_trial_features_m = pd.read_csv(data_folder_name + '/patterns_and_features/all_trial_features.csv')
all_trial_features_valid_m = all_trial_features_m[(all_trial_features_m['t_last_vis']<50) & (all_trial_features_m['hitting_arena_edge']==False)].reset_index()
median_values_m = all_trial_features_valid_m.median(axis=0)
all_trial_features_valid_m = all_trial_features_m[(all_trial_features_m['t_last_vis'] < 50) & (all_trial_features_m['hitting_arena_edge']==False)].reset_index()


In [ ]:
all_trial_features_valid = data_item.all_trial_features[(data_item.all_trial_features['t_last_vis'] < 50) & (data_item.all_trial_features['hitting_arena_edge']==False)].reset_index()
plot_statistics.plot_feature_histograms_for_monkey_and_agent(all_trial_features_valid_m, all_trial_features_valid)

# Plot RL & monkey side_by_side

In [ ]:
stop!

In [ ]:
model_folder_name = "multiff_analysis/RL_models/SB3_stored_models/all_agents/A0.2_O4_ff2_memory3/"
env_kwargs = {"num_obs_ff": 5}
env = env.CollectInformation(**env_kwargs)
env.reset()


# For direct training
sac_model = SAC("MlpPolicy", 
            env,
            gamma=0.995,
            learning_rate=0.0015,
            batch_size=1024,
            target_update_interval=50,
            buffer_size=1000000,
            learning_starts=10000,
            train_freq=10,
            ent_coef='auto',
            policy_kwargs=dict(activation_fn=nn.Tanh, net_arch=[128, 128])
                )
path = os.path.join(model_folder_name, 'best_model.zip')
sac_model = sac_model.load(path,env=env) 


with general_utils.HiddenPrints():
  # Note: must use env(Simulated)

  agent_dt = 0.25
  num_trials = 2

  plotting_params = {"show_stops": True,
                    "show_believed_target_positions": True,
                    "show_reward_boundary": True,
                    "show_connect_path_ff": True,
                    "show_scale_bar": True,
                    "hitting_arena_edge_ok": True,
                    "trial_too_short_ok": True}

  for currentTrial in [12, 69, 138, 221, 235, 259, 263, 265, 299, 393, 496, 523, 556, 601, 666, 698, 760, 805, 808, 930, 946, 955, 1002, 1003]:
      info_of_agent, plot_whole_duration, rotation_matrix, num_imitation_steps_monkey, num_imitation_steps_agent = process_agent_data.find_corresponding_info_of_agent(info_of_monkey, currentTrial, num_trials, sac_model, agent_dt, LSTM=False, env_kwargs=env_kwargs)

      with general_utils.initiate_plot(20,20,400):
          additional_plots.PlotSidebySide(plot_whole_duration = plot_whole_duration,
                          info_of_monkey = info_of_monkey,
                          info_of_agent = info_of_agent,  
                          num_imitation_steps_monkey = num_imitation_steps_monkey,
                          num_imitation_steps_agent = num_imitation_steps_agent,                
                          currentTrial = currentTrial,
                          num_trials = num_trials, 
                          rotation_matrix = rotation_matrix,              
                          plotting_params = plotting_params,
                          )

# Plot RL parameter vs outcome

Should I use marginal or conditional?

Should I have 2 variables or 3 variables?

Or should I just try all of the above?

Or, can I make an interactive plot?

In [ ]:
overall_folder = 'multiff_analysis/RL_models/SB3_stored_models/all_agents/gen_5/'

In [ ]:
feature_medians_record = pd.read_csv(overall_folder + 'feature_medians_record.csv').drop(["Unnamed: 0"], axis=1)
feature_means_record = pd.read_csv(overall_folder + 'feature_means_record.csv').drop(["Unnamed: 0"], axis=1)
pattern_frequencies_record = pd.read_csv(overall_folder +'pattern_frequencies_record.csv').drop(["Unnamed: 0"], axis=1)

In [ ]:
df_of_interest = pattern_frequencies_record

parameter_columns = ['v_noise_std', 'w_noise_std', 'ffr_noise_scale', 'num_obs_ff', 'max_in_memory_time']
outcome_columns = np.setdiff1d(df_of_interest.columns, parameter_columns)
#plot_statistics.plot_correlations_in_record(df=df_of_interest, parameter_columns=parameter_columns, outcome_columns=outcome_columns)
outcome_columns

## Marking what works

In [ ]:
# parameters_record = feature_medians_record[['v_noise_std', 'w_noise_std', 'ffr_noise_scale', 'num_obs_ff', 'max_in_memory_time']].copy()
# parameters_record['working'] = 9 # I will change this manually; 2 meanings working well; 1 means somewhat working; 0 meanings not working
# parameters_record = parameters_record.sort_values(by=['v_noise_std', 'w_noise_std', 'ffr_noise_scale', 'num_obs_ff', 'max_in_memory_time']).reset_index(drop=True)
# parameters_record.to_csv(overall_folder + 'parameters_record.csv')

In [ ]:
parameters_record = pd.read_csv(overall_folder + 'parameters_record.csv').drop(["Unnamed: 0"], axis=1)
parameters_record

In [ ]:
parameter_columns = ['v_noise_std', 'w_noise_std', 'ffr_noise_scale', 'num_obs_ff', 'max_in_memory_time']
outcome_columns = parameter_columns.copy()
plot_statistics.plot_correlations_in_record(parameters_record, parameter_columns=parameter_columns, outcome_columns=outcome_columns,\
                            color_column='working')

## interactions BETWEEN the parameter columns

In [ ]:
parameter_columns = ['v_noise_std', 'w_noise_std', 'ffr_noise_scale', 'num_obs_ff', 'max_in_memory_time']
outcome_columns = parameter_columns.copy()
plot_statistics.plot_correlations_in_record(feature_medians_record, parameter_columns=parameter_columns, outcome_columns=outcome_columns)

## medians

In [ ]:
feature_outcome_columns = ['abs_angle_last_vis', 'd_last_vis', 'n_ff_in_a_row', 'num_stops',
'num_stops_near_target', 'num_stops_since_last_vis', 't', 't_last_vis']
plot_statistics.plot_correlations_in_record(feature_medians_record, outcome_columns=feature_outcome_columns)

## means

In [ ]:
feature_outcome_columns = ['abs_angle_last_vis', 'd_last_vis', 'n_ff_in_a_row', 'num_stops',
'num_stops_near_target', 'num_stops_since_last_vis', 't', 't_last_vis']
plot_statistics.plot_correlations_in_record(feature_means_record, outcome_columns=feature_outcome_columns)

## frequencies

In [ ]:
pattern_outcome_columns = ['ff_capture_rate', 'stop_success_rate', 'two_in_a_row', 'waste_cluster_around_target', 
 'disappear_latest', 'visible_before_last_one', 'try_a_few_times', 'give_up_after_trying', 'ignore_sudden_flash']
 
plot_statistics.plot_correlations_in_record(pattern_frequencies_record, outcome_columns=pattern_outcome_columns)

## Try using a dummy dataframe

In [ ]:
dummy_dict = {}
for column in feature_medians_record.columns:
    dummy_dict[column] = np.random.rand(10)
dummy_dict
dummy_df = pd.DataFrame(dummy_dict)


plot_statistics.plot_correlations_in_record(df=dummy_df, parameter_columns=parameter_columns, outcome_columns=feature_outcome_columns)

# Appendix

In [ ]:
# Retrieve more agent data
all_trial_features_lstm = pd.read_csv('multiff_analysis/RL_models/LSTM_stored_models/all_agents/gen_0/LSTM_Aug_1_24/patterns_and_features/all_trial_features.csv')
all_trial_features_valid_lstm = all_trial_features_lstm[(all_trial_features_lstm['t_last_vis']<50) & (all_trial_features_lstm['hitting_arena_edge']==False)].reset_index()
median_values_lstm = all_trial_features_valid_lstm.median(axis=0)

In [ ]:
# what percentage of points in monkey_information['time'].diff() is below 0.015?
t_diff = monkey_information['time'].diff()
np.where(t_diff < 0.015)[0].shape[0]/t_diff.shape[0]

In [ ]:
# find trials where two ff within 100 cm of each other appear at the same time
trials_for_planning = []
for trial in ff_dataframe_visible.target_index.unique():
    ff_subset = ff_dataframe_visible[ff_dataframe_visible.target_index == trial]
    ff_counting = ff_subset.groupby('point_index').count()
    ff_counting = ff_counting[ff_counting.ff_index > 1]
    point_index = ff_counting.point_index
    #ff_subset = ff_subset[ff_subset.point_index.isin(point_index)]
    # for each point index, see if the ff are within 100 of each other
    for point_index in relevant_point_index:
        current_ff = ff_subset[ff_subset.point_index == point_index]
        # find if they're within 100 cm of each other
        # ... and then I decided to use find_point_vs_cluster
    if len(ff_subset) > 0:
        trials_for_planning.append(trial)


## angle vs time
target_angle_to_boundary_last_vis VS. time_since_target_cluster_last_vis

In [ ]:
# Subset data
target_clust_sub = target_clust_last_vis_df[target_clust_last_vis_df['time_since_last_vis'] < 7]

# Convert angle to degrees
angles_deg = target_clust_sub['abs_last_vis_ang_to_bndry'] * 180 / np.pi

# Plot with regression fit
plt.figure(figsize=(7,7))
sns.regplot(
    x=target_clust_sub['time_since_last_vis'], 
    y=angles_deg, 
    scatter_kws={'s': 10},   # scatter point size
    line_kws={'color':'red'} # regression line color
)
plt.xlabel('Time since last visible (s)')
plt.ylabel('Absolute angle to boundary when last visible (deg)')
plt.title('Regression of Angle When Last Visible vs Time Since Last Visible')
plt.show()


## Inspect angles to boundary

Since target cluster became last visible, has the angle to boundary of the target ever become greater than 40 degrees?

Note: here we use angle to reward boundary, not angle to visible boundary

In [ ]:
ff_indices_of_each_cluster = cluster_analysis.find_alive_target_clusters(ff_real_position_sorted, ff_caught_T_new, ff_life_sorted, max_distance=50)

In [ ]:
reload(specific_utils)
problematic_trials = []
# for each target_index
for index, row in target_clust_last_vis_df.iterrows():
    if row.time_since_last_vis < 3:
        target_index = row.target_index
        #ff_indices = ff_indices_of_each_cluster[target_index]
        ff_indices = np.array([target_index]) # I realized that ultimately ff_indices can only be target_index, cause otherwise there will likely be big angles
        time_interval = [row.last_vis_time, row.caught_time]
        # in this time interval, calculate all ff_angle_boundary of ff_indices and see if any is greater than 45
        monkey_information_subset = monkey_information[(monkey_information.time >= time_interval[0]) & (monkey_information.time <= time_interval[1])]
        monkey_xy_relevant = monkey_information_subset[['monkey_x', 'monkey_y']].values
        monkey_angles_relevant = monkey_information_subset['monkey_angle'].values
        for ff_index in ff_indices:
            ff_distance_relevant = np.linalg.norm(monkey_xy_relevant-ff_real_position_sorted[ff_index], axis=1)
            angles_to_ff = specific_utils.calculate_angles_to_ff_centers(ff_x=ff_real_position_sorted[ff_index, 0], ff_y=ff_real_position_sorted[ff_index, 1], mx=monkey_xy_relevant[:, 0], my=monkey_xy_relevant[:, 1], m_angle=monkey_angles_relevant)
            angles_to_boundaries = specific_utils.calculate_angles_to_ff_boundaries(angles_to_ff=angles_to_ff, distances_to_ff=ff_distance_relevant, 
                                                                                ff_radius=10)      
            angles_to_boundaries = angles_to_boundaries*180/math.pi
            big_angles_to_boundaries_indices = np.where(np.abs(angles_to_boundaries) >= 40)[0]
            # if the angle only appears after the monkey passes the ff, then it's not a problem
            if len(big_angles_to_boundaries_indices) > 0:
                big_angles_to_boundaries = angles_to_boundaries[big_angles_to_boundaries_indices]
                if not np.any(np.abs(angles_to_boundaries[:big_angles_to_boundaries_indices[0]]) < 30):
                    problematic_trials.append(target_index)
                    print('target_index: ', target_index)
                    print('ff_index: ', ff_index, 'angles_to_boundaries: ', big_angles_to_boundaries)
                    print('all angles_to_boundaries: ', angles_to_boundaries)

In [ ]:
reload(plot_behaviors_utils)
num_trials = 2
plt.rcParams["figure.figsize"] = (10, 10)
for currentTrial in problematic_trials[3:5]:
    plot_behaviors_utils.plot_a_trial(currentTrial, 2, ff_caught_T_new, PlotTrials_args, additional_kwargs=None)

# Planning

## find point_vs_cluster

In [ ]:
# keep only the ff that monkey has passed by closely
ff_dataframe_sifted = pd.DataFrame()
for time in range(math.floor(ff_dataframe.time.min()), math.ceil(ff_dataframe.time.max()), 15):
    duration = [time, time+20] # note, we try to create some overlaps between intervals
    ff_dataframe_portion_to_keep = ff_dataframe_utils.keep_only_ff_that_monkey_has_passed_by_closely(ff_dataframe, duration=duration, max_distance_to_ff=100)
    ff_dataframe_sifted = pd.concat([ff_dataframe_sifted, ff_dataframe_portion_to_keep])
ff_dataframe_sifted = ff_dataframe_sifted.drop_duplicates()

ff_dataframe_sifted_visible = ff_dataframe_sifted[ff_dataframe_sifted['visible'] == True]


In [ ]:
filepath = os.path.join(data_item.patterns_and_features_folder_path, 'point_vs_cluster.csv')
if exists(filepath): 
    point_vs_cluster = pd.read_csv(filepath).drop(["Unnamed: 0"], axis=1)     
    point_vs_cluster = point_vs_cluster.set_index(['point_index', 'cluster_label']) 
else:
    point_vs_cluster = cluster_analysis.make_point_vs_cluster(ff_dataframe_sifted_visible, max_ff_distance_from_monkey = 400, max_cluster_distance = 50, max_time_past = 0, 
                            print_progress = True, data_folder_name = None)
    point_vs_cluster['time'] = monkey_information.loc[point_vs_cluster['point_index'], 'time'].values
    point_vs_cluster.to_csv(filepath)
    point_vs_cluster = point_vs_cluster.set_index(['point_index', 'cluster_label'])

## Plot

In [ ]:

plt.rcParams['figure.figsize'] = [10, 10]                     
max_plot_to_make = 3
num_trials = 1
mean_interval_between_plots = 0.3
prev_plot_time = -99
plot_counter = 0
 
 # use trials after 330
point_vs_cluster_subset = point_vs_cluster[point_vs_cluster['target_index'] >= 330]   

#print("Note: only showing the ff that monkey has passed by closely")  
# when checking the code later, I don't see how the sentence above was implemented. Maybe I need to run the code to see what happened

# iterate through rows in point_vs_cluster
for (point_index, cluster_label), row in point_vs_cluster_subset.iterrows():

    # category_item.plot_trajectories(trials=[row.target_index.astype('int')])
    # break

    # also make sure that two plots are not too close to each other in time
    if row.time > prev_plot_time + mean_interval_between_plots:
        prev_plot_time = row.time
        ff_indices = point_vs_cluster[point_vs_cluster.index == (point_index, cluster_label)].ff_index.to_numpy()
        plotting_kwargs_temp = plotting_kwargs.copy()
        plotting_kwargs_temp['indices_of_ff_to_mark_2nd_kind'] = ff_indices
        plotting_kwargs_temp['point_indices_to_be_marked_2nd_kind'] = point_index

        
        trial = row.target_index.astype('int')
        duration = [data_item.ff_caught_T_new[trial-num_trials], max(data_item.ff_caught_T_new[trial], row.time+4)]

        returned_info = plot_trials.PlotTrials(
                    duration, 
                    *data_item.PlotTrials_args,
                    **plotting_kwargs_temp,
                    null_agent_starting_time = row.time,
                    currentTrial = trial,
                    num_trials = num_trials,    
                    #null_arc_info_for_plotting              
                    )
        plt.show()
        

        plot_counter += 1
        if plot_counter >= max_plot_to_make:
            break

##  Plot with eye positions

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]                     
max_plot_to_make = 2
num_trials = 1
mean_interval_between_plots = 0.3
prev_plot_time = -99
plot_counter = 0
 
 # use trials after 330 
point_vs_cluster_subset = point_vs_cluster[point_vs_cluster['target_index'] >= 330]   

print("Note: only showing the ff that monkey has passed by closely")   
# when checking the code later, I don't see how the sentence above was implemented. Maybe I need to run the code to see what happened

# iterate through rows in point_vs_cluster
for (point_index, cluster_label), row in point_vs_cluster_subset.iterrows():

    # category_item.plot_trajectories(trials=[row.target_index.astype('int')])
    # break
    if row.time > prev_plot_time + mean_interval_between_plots:
        prev_plot_time = row.time
        ff_indices = point_vs_cluster[point_vs_cluster.index == (point_index, cluster_label)].ff_index.to_numpy()
        plotting_kwargs_temp = plotting_kwargs.copy()
        plotting_kwargs_temp['indices_of_ff_to_mark_2nd_kind'] = ff_indices
        plotting_kwargs_temp['point_indices_to_be_marked_2nd_kind'] = point_index
        plotting_kwargs_temp['show_eye_positions'] = True
        plotting_kwargs_temp['show_connect_path_eye_positions'] = True
        
        
        
        trial = row.target_index.astype('int')
        duration = [data_item.ff_caught_T_new[trial-num_trials], max(data_item.ff_caught_T_new[trial], row.time+4)]

        returned_info = plot_trials.PlotTrials(
                    duration, 
                    *data_item.PlotTrials_args,
                    **plotting_kwargs_temp,
                    null_agent_starting_time = row.time,
                    currentTrial = trial,
                    num_trials = num_trials,                   
                    )
        plt.show()
        

        plot_counter += 1
        if plot_counter >= max_plot_to_make:
            break

# Debug

In [ ]:
#!pip install -Uqq ipdb
import ipdb
%pdb on